In [50]:
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests

In [51]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [52]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [53]:

html = browser.html

# Parse HTML
soup = BeautifulSoup(html, 'html.parser')


# collect the latest News Title and Paragraph Text and assign variables
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='article_teaser_body').text


# Display scrapped data 
print(news_title)
print(news_p)

NASA's Mars 2020 Will Hunt for Microscopic Fossils
A new paper identifies a ring of minerals at the rover's landing site that are ideal for fossilizing microbial life.


In [54]:
# go to Mars Space Images
image_url_featured = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url_featured)

In [55]:
html_image = browser.html

# Parse HTML
soup = BeautifulSoup(html_image, 'html.parser')

# Retrieve background-image url from style tag 
featured_image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

# Website Url 
main_url = 'https://www.jpl.nasa.gov'

# complete url string for this image
featured_image_url = main_url + featured_image_url

# Display full link to featured image
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18640-1920x1200.jpg'

In [56]:
# go to Mars Weather Twitter through splinter module
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [58]:
#Parse HTML
html_weather = browser.html
soup = BeautifulSoup(html_weather, 'html.parser')

# Find all elements that contain tweets
new_tweets = soup.find_all('div', class_='js-tweet-text-container')

# Loop through returned results
for tweet in new_tweets:
    # Error handling
    try:
        # Identify and return title of listing
        mars_weather_tweet = tweet.find('p').text

        # Run only if sol, and pressure are available
        if 'Sol' and 'pressure' in mars_weather_tweet:
            # Print results
            print(mars_weather_tweet)
            #save output as variable mars_weather
            mars_weather_tweet = mars_weather
            break
        else:
            pass

    except Exception as e:
        print(e)

InSight sol 340 (2019-11-10) low -101.0ºC (-149.8ºF) high -24.4ºC (-12.0ºF)
winds from the SSE at 5.3 m/s (11.9 mph) gusting to 20.4 m/s (45.6 mph)
pressure at 6.90 hPapic.twitter.com/5m82oQ9M6z


In [59]:
mars_weather

'InSight sol 340 (2019-11-10) low -101.0ºC (-149.8ºF) high -24.4ºC (-12.0ºF)\nwinds from the SSE at 5.3 m/s (11.9 mph) gusting to 20.4 m/s (45.6 mph)\npressure at 6.90 hPapic.twitter.com/5m82oQ9M6z'

In [60]:
# go to Mars facts url 
facts_url = 'http://space-facts.com/mars/'

#read facts into pandas
mars_facts = pd.read_html(facts_url)

mars_df = mars_facts[0]

mars_df.columns = ['Description','Value']

# Set the index to the `Description` column without row indexing
#mars_df.set_index('Description', inplace=True)

# convert to html string
mars_df.to_html()

# Here's our added param..
data = mars_df.to_dict(orient='records')  

# Display mars_df
mars_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [61]:
# go to hemispheres website
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)

In [62]:
# Parse HTML
html_hemispheres = browser.html
soup = BeautifulSoup(html_hemispheres, 'html.parser')

# Retreive all items that contain mars hemispheres information
items = soup.find_all('div', class_='item')

# Create list for urls 
hemisphere_image_urls = []

hemispheres_main_url = 'https://astrogeology.usgs.gov'

# Loop through items stored
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # Parse HTML for individual hemisphere website 
    partial_img_html = browser.html
    soup = BeautifulSoup( partial_img_html, 'html.parser')
    
    # get full source 
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    
    # put info into list
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [66]:
# Import Dependencies 
from flask import Flask, render_template, redirect 
from flask_pymongo import PyMongo
import scrape


# Create an instance of Flask app
app = Flask(__name__)


mongo = PyMongo(app, uri="mongodb://localhost:27017/mars")



@app.route('/')
def index():
    mars_info = mongo.db.mars_info.find_one()
    return render_template('index.html', mars_info=mars_info)




@app.route("/scrape")
def scrape(): 

    # Run scrapped functions
    mars_info = mongo.db.mars
    mars_data = scrape.scrape_mars_news()
    mars_data = scrape.scrape_mars_images()
    mars_data = scrape.scrape_mars_facts()
    mars_data = scrape.scrape_mars_weather()
    mars_data = scrape.scrape_mars_hemi()
    mars_info.update({}, mars_data, upsert=True)

    return redirect("/", code=302)

if __name__ == "__main__": 
    app.run(debug= True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 120